<h1>Part 03 - Data Management</h1>

<h2>Exercise 02 - Connect to an Open API</h2>


<font size="3">
Goal of this section is to integrate your first API data, starting with an easy example of publicly open and free API that doesn't require any credentials to connect.

With this exercise you will be able to code all the flow, from the request to the transformation of the JSON format into a CSV that is then persisted localy.
</font>

### Get the Data with a request

In [1]:
import requests
import os
import csv
import locale
locale.setlocale(locale.LC_NUMERIC, "en_US.UTF-8")

'en_US.UTF-8'

Explore the [requests](https://pypi.org/project/requests/) documentation to call the https://api.coindesk.com/v1/bpi/currentprice.json API

In [2]:
response = requests.get("https://api.coindesk.com/v1/bpi/currentprice.json", headers={"Accept": "application/json"})
response.status_code

200

Get the JSON from the response

In [3]:
data = response.json()
data

{'time': {'updated': 'Mar 9, 2023 15:12:00 UTC',
  'updatedISO': '2023-03-09T15:12:00+00:00',
  'updateduk': 'Mar 9, 2023 at 15:12 GMT'},
 'disclaimer': 'This data was produced from the CoinDesk Bitcoin Price Index (USD). Non-USD currency data converted using hourly conversion rate from openexchangerates.org',
 'chartName': 'Bitcoin',
 'bpi': {'USD': {'code': 'USD',
   'symbol': '&#36;',
   'rate': '21,724.7274',
   'description': 'United States Dollar',
   'rate_float': 21724.7274},
  'GBP': {'code': 'GBP',
   'symbol': '&pound;',
   'rate': '18,153.0084',
   'description': 'British Pound Sterling',
   'rate_float': 18153.0084},
  'EUR': {'code': 'EUR',
   'symbol': '&euro;',
   'rate': '21,163.0563',
   'description': 'Euro',
   'rate_float': 21163.0563}}}

### Save the results into variables

Save the folowing values extracted in the JSON in dedicated variable :  
- time = Time when the data was updated in ISO format
- usd_rate = Rate of the BTC in USD in FLOAT format
- eur_rate = Rate of the BTC in EUR in FLOAT format
- gbp_rate = Rate of the BTC in GBP in FLOAT format

In [4]:
time = data["time"]["updatedISO"]
time

'2023-03-09T15:12:00+00:00'

In [5]:
usd_rate = locale.atof(data["bpi"]["USD"]["rate"])
usd_rate

21724.7274

In [6]:
eur_rate = locale.atof(data["bpi"]["EUR"]["rate"])
eur_rate

21163.0563

In [7]:
gbp_rate = locale.atof(data["bpi"]["GBP"]["rate"])
gbp_rate

18153.0084

**Take a time to think :** what happens here if the JSON format coming from the API change ?

Everything will break.

### Save those values in a CSV file

Create a CSV file, with the variable names as headers, and the values of those variable inserted in the first line. Save this file locally in the data folder named `data/today_BTC_rates.csv`

In [8]:
headers = ["time", "usd_rate", "eur_rate", "gbp_rate"]
rows = [
    {"time": time, "usd_rate": usd_rate, "eur_rate": eur_rate, "gbp_rate": gbp_rate},
]

with open(r"data/today_BTC_rates.csv", mode="w", encoding="UTF8", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    writer.writerows(rows)

Test to read the result with the code below :

In [9]:
import pandas as pd

btc_rates = pd.read_csv("data/today_BTC_rates.csv")
btc_rates.head()

,time,usd_rate,eur_rate,gbp_rate
0,2023-03-09T15:12:00+00:00,21724.7274,21163.0563,18153.0084


Now everytime your run the code above, you should be able to refresh the CSV based on todays' BTC rates and integrate this information in any system workflow where it's needed.

### Append to code to the local CSV

But now what if we don't want to overwrite our file to keep an history of this BTC rates in the CSV file, just appending the last line of value to our CSV ?  
Code the logic that will : 
- Create the `data/BTC_rates.csv` with the header if it doesn't exist
- Append our line of value to the current file

In [10]:
headers = ["time", "usd_rate", "eur_rate", "gbp_rate"]

with open(r"data/BTC_rates.csv", mode="a", encoding="UTF8", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    
    # Write the header if the file is empty
    file.seek(0, os.SEEK_END)
    if not file.tell():
        writer.writeheader()
    
    writer.writerow({"time": time, "usd_rate": usd_rate, "eur_rate": eur_rate, "gbp_rate": gbp_rate})

Run the whole notebook several time and check if your CSV file is appending the new line of data with the following command :

In [11]:
btc_rates = pd.read_csv(r"data/BTC_rates.csv")
btc_rates.head()

,time,usd_rate,eur_rate,gbp_rate
0,2023-03-09T14:54:00+00:00,21713.7778,21152.3898,18143.8590
1,2023-03-09T15:11:00+00:00,21740.5969,21178.5155,18166.2689
2,2023-03-09T15:11:00+00:00,21740.5969,21178.5155,18166.2689
3,2023-03-09T15:12:00+00:00,21724.7274,21163.0563,18153.0084


### Wrapping up

Congratulations ! You are now able to build an history of BTC to USD / EUR / GBP data using an Open API. This code can be scheduled every day with Prefact to update your CSV file and to slowly be able to build a daily rates datasource. In the next steps we are going to explore how to connect to an API that is protected with an Authentication system.